In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("Biostrings")

library(Biostrings)
library(ggplot2)

In [2]:
rarefaction_curve <- function(fasta_file, num_repeats) {
  sequences <- readDNAStringSet(fasta_file)
  num_sequences <- length(sequences)
  all_unique_counts <- matrix(0, nrow = num_repeats, ncol = num_sequences)

  for (repeat_idx in 1:num_repeats) {
  set.seed(repeat_idx)  
  randomized_seq <- sample(sequences)

  unique_sequences <- character(0)
  for (i in seq_along(randomized_seq)) {
    unique_sequences <- unique(c(unique_sequences, as.character(randomized_seq[i])))
    all_unique_counts[repeat_idx, i] <- length(unique_sequences)
  }
  }

  average_unique_counts <- colMeans(all_unique_counts)
  std_dev_unique_counts <- apply(all_unique_counts, 2, sd) 
  
  rarefaction_data <- data.frame(
  num_sequences_analyzed = 1:num_sequences,
  num_unique_sequences = average_unique_counts,
  lower_bound = average_unique_counts - std_dev_unique_counts,
  upper_bound = average_unique_counts + std_dev_unique_counts
  )
  
  rarefaction_data$slope <- c(NA, diff(rarefaction_data$num_unique_sequences) / diff(rarefaction_data$num_sequences_analyzed))
  
  max_value <- max(c(rarefaction_data$num_sequences_analyzed, rarefaction_data$num_unique_sequences))

  p1 <- ggplot(rarefaction_data, aes(x = num_sequences_analyzed, y = num_unique_sequences)) +
  geom_line() +
  geom_ribbon(aes(ymin = lower_bound, ymax = upper_bound), alpha = 0.2) +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
  labs(
    title = "Rarefaction Curve on proviral sequences",
    x = "Number of Sequences Analyzed",
    y = "Number of Unique Sequences"
  ) +
  theme_bw() +
  theme(axis.text = element_text(size = 14), axis.title = element_text(size = 14)) +
  coord_equal() +
  xlim(0, max_value) +
  ylim(0, max_value)

  p2 <- ggplot(rarefaction_data, aes(x = num_sequences_analyzed, y = slope)) +
  geom_line() +
  labs(
    title = "Rarefaction Slope on proviral sequences",
    x = "Number of Sequences Analyzed",
    y = "Slope"
  ) +
  theme_bw() +
  theme(axis.text = element_text(size = 14), axis.title = element_text(size = 14)) +
  ylim(0, 1) +
  xlim(0, max_value)

  combined_plot <- grid.arrange(p1, p2, ncol = 2)
  ggsave("rarefaction_curve.png", plot = combined_plot, width = 20, height = 10, units = "in", dpi = 300)

}

In [ ]:
fasta_file <- #Insert FASTA here (e.g. "sequences.fasta") 
num_repeats <- 1000
rarefaction_curve(fasta_file, num_repeats)